In [3]:
#!pip install faiss-cpu
#!pip install sentence-transformers

In [1]:
import pandas as pd
import numpy as np
import faiss
import pickle
from sentence_transformers import SentenceTransformer

/Users/harshilgohil/python/Recommender system/Movie_recommender_system/venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
movies = pd.read_csv("tmdb_movies.csv")
movies.head()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


In [3]:
movies.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [4]:
movies = movies[['id','title','overview','genre']]
movies.head()

,id,title,overview,genre
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"Drama,Crime"
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","Comedy,Drama,Romance"
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","Drama,Crime"
3,424,Schindler's List,The true story of how businessman Oskar Schind...,"Drama,History,War"
4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...,"Drama,Crime"


In [5]:
movie_data = movies['title'] + movies['overview'] + movies['genre']
movie_data[0]

'The Shawshank RedemptionFramed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.Drama,Crime'

In [31]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(movie_data)
vectors.shape

(10000, 768)

In [6]:
encoder = SentenceTransformer("all-mpnet-base-v2")

In [32]:
dim = vectors.shape[1]

In [33]:
dim

768

In [7]:
dim = 768
index = faiss.IndexFlatL2(dim) 

In [35]:
index.add(vectors)

In [37]:
with open("faiss_index.pkl","wb") as f:
    pickle.dump(index,f)

In [38]:
with open("movie.pkl","wb") as f:
    pickle.dump(movie_d,f)

In [8]:
query = "The Godfather"
vec = encoder.encode(query)
vec.shape

(768,)

In [9]:
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [41]:
distances, ind = index.search(svec, k=6)

In [42]:
ind

array([[1611,    2,    4, 6239, 2188,  767]])

In [43]:
ind[0]

array([1611,    2,    4, 6239, 2188,  767])

In [44]:
movies.loc[ind[0],'title']

1611    The Godfather: Part III
2                 The Godfather
4        The Godfather: Part II
6239                 The Family
2188              The Outsiders
767                 The Traitor
Name: title, dtype: object

In [45]:
recomended_5 = movies.loc[ind[0],'title']
for i in recomended_5:
    print(i)

The Godfather: Part III
The Godfather
The Godfather: Part II
The Family
The Outsiders
The Traitor


In [46]:
movies.loc[ind[0],'id']

1611       242
2          238
4          240
6239    112205
2188       227
767     575452
Name: id, dtype: int64

In [36]:
with open("faiss_index.pkl", "rb") as f:
    vectorstore = pickle.load(f)

In [37]:
distances, indx = vectorstore.search(svec, k=6)

In [38]:
distances[0]

array([0.62686515, 0.6943517 , 0.7489637 , 0.8851248 , 0.8901727 ,
       0.924353  ], dtype=float32)

In [40]:
indx[0]

array([1611,    2,    4, 6239, 2188,  767])

In [41]:
recomended_mv = movies.loc[indx[0],'title']
for i in recomended_mv:
    print(i)

The Godfather: Part III
The Godfather
The Godfather: Part II
The Family
The Outsiders
The Traitor
